In [1]:
!git clone https://github.com/teavanist/MNIST-JPG

Cloning into 'MNIST-JPG'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 29 (delta 3), reused 2 (delta 0), pack-reused 20
Unpacking objects: 100% (29/29), done.


In [2]:
import os
from glob import glob
from subprocess import run

for gz in glob(os.path.join('/content', '*.gz')):
    after = gz.replace('-idx', '.idx')
    os.rename(gz, after)
    run(['gunzip', after])

In [3]:
mnist_dir = '/content/data_mnist'
os.makedirs(mnist_dir, exist_ok=True)

!python /content/MNIST-JPG/mnist_jpg.py "$mnist_dir"

writing /content/data_mnist/testing/5/9982.jpg
writing /content/data_mnist/testing/0/9983.jpg
writing /content/data_mnist/testing/1/9984.jpg
writing /content/data_mnist/testing/2/9985.jpg
writing /content/data_mnist/testing/3/9986.jpg
writing /content/data_mnist/testing/4/9987.jpg
writing /content/data_mnist/testing/5/9988.jpg
writing /content/data_mnist/testing/6/9989.jpg
writing /content/data_mnist/testing/7/9990.jpg
writing /content/data_mnist/testing/8/9991.jpg
writing /content/data_mnist/testing/9/9992.jpg
writing /content/data_mnist/testing/0/9993.jpg
writing /content/data_mnist/testing/1/9994.jpg
writing /content/data_mnist/testing/2/9995.jpg
writing /content/data_mnist/testing/3/9996.jpg
writing /content/data_mnist/testing/4/9997.jpg
writing /content/data_mnist/testing/5/9998.jpg
writing /content/data_mnist/testing/6/9999.jpg


In [4]:
from tqdm.auto import tqdm
from PIL import Image

for f in ['train', 'val']:
    for i in range(10):
        os.makedirs(f'/content/data/{f}/{i}', exist_ok=True)

for f_in in ['training', 'testing']:
    print(f'{f_in} folder')
    for j in tqdm(range(10)):
        dir = f'/content/data_mnist/{f_in}/{j}'
        for image in os.listdir(dir):
            path_in = os.path.join(dir, image)
            I = Image.open(path_in).resize((224, 224)).convert('RGB')
            if f_in == 'training':
                f_out = 'train'
            else:
                f_out = 'val'
            path_out = f'/content/data/{f_out}/{j}/{image}'
            I.save(path_out)

training folder



testing folder


In [5]:
import cv2
import numpy as np
from pathlib import Path

path_images = Path('/content/data/train')
num_images = len(list(path_images.glob('**/*.jpg')))
mean = 0
std = 0
for image in tqdm(path_images.glob('**/*.jpg'), total=num_images):
    image = str(image)
    img = cv2.imread(image)
    mean += np.mean(img[:, :, 0]) / num_images
    std += np.std(img[:, :, 0]) / num_images
mean, std

(34.51498935015433, 73.50595929282494)

In [6]:
!pip install timm
!git clone https://github.com/yitu-opensource/T2T-ViT

     |████████████████████████████████| 245kB 11.2MB/s 
Cloning into 'T2T-ViT'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 32 (delta 17), reused 15 (delta 7), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [7]:
import os
os.chdir('/content/T2T-ViT')

PATH_TO_DATA = '/content/data'

!bash distributed_train.sh 1 "$PATH_TO_DATA"\
    --model T2t_vit_t_14\
    --batch-size 32\
    --num-classes 10\
    --img-size 224\
    --mean 34.515\
    --std 73.506\
    --lr 1e-4\
    --epochs 1

Training with a single process on 1 GPUs.
adopt transformer encoder for tokens-to-token
Model T2t_vit_t_14 created, param count: 21159920
Data processing configuration for current model + dataset:
	input_size: (3, 224, 224)
	interpolation: bicubic
	mean: (34.515, 34.515, 34.515)
	std: (73.506, 73.506, 73.506)
	crop_pct: 0.9
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 11
Train: 0 [   0/1875 (  0%)]  Loss:  2.332949 (2.3329)  Time: 2.328s,   13.75/s  (2.328s,   13.75/s)  LR: 1.000e-06  Data: 1.470 (1.470)
Train: 0 [  50/1875 (  3%)]  Loss:  2.300109 (2.3300)  Time: 0.430s,   74.46/s  (0.468s,   68.33/s)  LR: 1.000e-06  Data: 0.004 (0.033)
Train: 0 [ 100/1875 (  5%)]  Loss:  2.323153 (2.3328)  Time: 0.429s,   74.59/s  (0.449s,   71.22/s)  LR: 1.000e-06  Data: 0.004 (0.019)
Train: 0 [ 150/1875 (  8%)]  Loss:  2.383229 (2.3309)  Time: 0.426s,   75.13/s  (0.443s,   72.27/s)  LR: 1.000e-06  Data: 0.004 (0.014)
Train: 0 [ 200/1875 ( 11%)]  Loss:  2.309027 (2.3269)  T

In [8]:
ckpt = '/content/T2T-ViT/output/train/20210204-051046-T2t_vit_t_14-224/checkpoint-10.pth.tar'
!cp "$ckpt" /content/drive/MyDrive

In [9]:
import os
os.chdir('/content/T2T-ViT')

PATH_TO_DATA = '/content/data'
PATH_TO_CHECKPOINT = '/content/drive/MyDrive/checkpoint-10.pth.tar'

!bash distributed_train.sh 1 "$PATH_TO_DATA"\
    --model T2t_vit_t_14\
    --initial-checkpoint "$PATH_TO_CHECKPOINT"\
    --batch-size 32\
    --num-classes 10\
    --img-size 224\
    --mean 34.515\
    --std 73.506\
    --lr 1e-4\
    --epochs 1

Training with a single process on 1 GPUs.
adopt transformer encoder for tokens-to-token
Loaded state_dict from checkpoint '/content/drive/MyDrive/checkpoint-10.pth.tar'
Model T2t_vit_t_14 created, param count: 21159920
Data processing configuration for current model + dataset:
	input_size: (3, 224, 224)
	interpolation: bicubic
	mean: (34.515, 34.515, 34.515)
	std: (73.506, 73.506, 73.506)
	crop_pct: 0.9
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 11
Train: 0 [   0/1875 (  0%)]  Loss:  2.122509 (2.1225)  Time: 2.443s,   13.10/s  (2.443s,   13.10/s)  LR: 1.000e-06  Data: 1.444 (1.444)
Train: 0 [  50/1875 (  3%)]  Loss:  1.598894 (1.8463)  Time: 0.431s,   74.19/s  (0.473s,   67.59/s)  LR: 1.000e-06  Data: 0.004 (0.033)
Train: 0 [ 100/1875 (  5%)]  Loss:  1.202552 (1.7073)  Time: 0.434s,   73.80/s  (0.455s,   70.40/s)  LR: 1.000e-06  Data: 0.004 (0.019)
Train: 0 [ 150/1875 (  8%)]  Loss:  1.586034 (1.7548)  Time: 0.432s,   74.03/s  (0.447s,   71.53/s)  LR: 1.000e

In [10]:
ckpt = '/content/T2T-ViT/output/train/20210204-082514-T2t_vit_t_14-224/model_best.pth.tar'
!cp "$ckpt" /content/drive/MyDrive